# Writing and Scheduling Vector Addition in TE for CPU


In [2]:
import tvm
import tvm.testing
from tvm import te
import numpy as np
import os
from tvm.contrib import cc
from tvm.contrib import utils

tgt = tvm.target.Target(target="llvm", host="llvm")

# Describing the Vector Computation

TVM adopts tensor semantics, with each intermediate result represented as a multi-dimensional array. The user needs to describe the computation rule that generates the tensors.

`n`: A symbolic variable to represent the shape. 
`A`: A placeholder Tensor, with given shape (n,)
`B`: A placeholder Tensor, with given shape (n,)

`C`: The result tensor, with a compute operation. 

`compute`: A computation, with the output conforming to the specified tensor shape and the computation to be performed at each position in the tensor defined by the lambda function. 

In [ ]:
n = te.var("n")
A = te.placeholder((n,), name="A")
B = te.placeholder((n,), name="B")
C = te.compute(A.shape, lambda i: A[i] + B[i], name="C")

# Create a Default Schedule for the Computation

While the above lines describe the computation rule, we can compute C in many different ways to fit different devices. For a tensor with multiple axes, you can choose which axis to iterate over first, or computations can be split across different threads. 

TVM requires that the user to provide a schedule, which is a description of how the computation should be performed. Scheduling operations within TE can change loop orders, split computations across different threads, and group blocks of data together, amongst other operations. An important concept behind schedules is that they only describe how the computation is performed, so different schedules for the same TE will produce the same result.

## Naive

TVM allows you to create a naive schedule that will compute C in by iterating in row major order.

The `tvm.lower` command will generate the Intermediate Representation (IR) of the TE, with the corresponding schedule. 

In [ ]:
s = te.create_schedule(C.op)
print(tvm.lower(s, [A, B, C], simple_mode=True))

## Leverage Parallel 

A schedule is a series of steps that are applied to an expression to transform it in a number of different ways. When a schedule is applied to an expression in TE, the inputs and outputs remain the same, but when compiled the implementation of the expression can change. 

We can apply the parallel schedule operation to our computation.



In [ ]:
s = te.create_schedule(C.op)
s[C].parallel(C.op.axis[0])
print(tvm.lower(s, [A, B, C], simple_mode=True))

## Leverage CPU Vectorization

Modern CPUs also have the ability to perform SIMD operations on floating point values, and we can apply another schedule to our computation expression to take advantage of this. 

Accomplishing this requires multiple steps: 
- First we have to split the schedule into inner and outer loops using the split scheduling primitive. The inner loops can use vectorization to use SIMD instructions using the vectorize scheduling primitive, then the outer loops can be parallelized using the parallel scheduling primitive. 
- Choose the split factor to be the number of threads on your CPU.

In [ ]:
s = te.create_schedule(C.op)

# This factor should be chosen to match the number of threads appropriate for
# your CPU. This will vary depending on architecture, but a good rule is
# setting this factor to equal the number of available CPU cores.
factor = 4

outer, inner = s[C].split(C.op.axis[0], factor=factor)
s[C].parallel(outer)
s[C].vectorize(inner)

# Complile the Schedule

We use `tvm.build` to create a function. The build function takes the schedule, the desired signature of the function (including the inputs and outputs) as well as target language we want to compile to.

In [ ]:
# For Naive
fadd = tvm.build(s, [A, B, C], tgt, name="fadd")
# For Parallelism
fadd_parallel = tvm.build(s, [A, B, C], tgt, name="fadd_parallel")
# For Vectorization
fadd_vector = tvm.build(s, [A, B, C], tgt, name="fadd_vector")

# Evaluate the Schedule

Run Schedule：

In [ ]:
dev = tvm.device(tgt.kind.name, 0)

n = 1024
a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), dev)
b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype), dev)
c = tvm.nd.array(np.zeros(n, dtype=C.dtype), dev)
fadd(a, b, c)
tvm.testing.assert_allclose(c.numpy(), a.numpy() + b.numpy())
fadd_parallel(a, b, c)
tvm.testing.assert_allclose(c.numpy(), a.numpy() + b.numpy())
fadd_vector(a, b ,c)
tvm.testing.assert_allclose(c.numpy(), a.numpy() + b.numpy())

a evaluation function:

In [ ]:
import timeit

np_repeat = 100
np_running_time = timeit.timeit(
    setup="import numpy\n"
    "n = 32768\n"
    'dtype = "float32"\n'
    "a = numpy.random.rand(n, 1).astype(dtype)\n"
    "b = numpy.random.rand(n, 1).astype(dtype)\n",
    stmt="answer = a + b",
    number=np_repeat,
)
print("Numpy running time: %f" % (np_running_time / np_repeat))


def evaluate_addition(func, target, optimization, log):
    dev = tvm.device(target.kind.name, 0)
    n = 32768
    a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), dev)
    b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype), dev)
    c = tvm.nd.array(np.zeros(n, dtype=C.dtype), dev)

    evaluator = func.time_evaluator(func.entry_name, dev, number=10)
    mean_time = evaluator(a, b, c).mean
    print("%s: %f" % (optimization, mean_time))

    log.append((optimization, mean_time))


log = [("numpy", np_running_time / np_repeat)]
evaluate_addition(fadd, tgt, "naive", log=log)
evaluate_addition(fadd_parallel, tgt, "parallel", log=log)
evaluate_addition(fadd_vector, tgt, "vector", log=log)

# Saving and Loading Compiled Modules

## Saving the compiled modules

The following code first performs the following steps:

- It saves the compiled host module into an object file.

- Then it saves the device module into a ptx file.

- `cc.create_shared` calls a compiler (gcc) to create a shared library

In [ ]:
tmp_path = "tmp"
os.makedirs(tmp_path, exist_ok=True)
fadd.save(os.path.join(tmp_path, "myadd.o"))

if tgt.kind.name == "cuda":
    fadd.imported_modules[0].save(os.path.join(tmp_path,"myadd.ptx"))
if tgt.kind.name == "rocm":
    fadd.imported_modules[0].save(os.path.join(tmp_path,"myadd.hsaco"))
if tgt.kind.name.startswith("opencl"):
    fadd.imported_modules[0].save(os.path.join(tmp_path,"myadd.cl"))
cc.create_shared(os.path.join(tmp_path,"myadd.so"), [os.path.join(tmp_path,"myadd.o")])
print(os.listdir(tmp_path))

## Load Compiled Module¶

We can load the compiled module from the file system and run the code. The following code loads the host and device module separately and links them together. We can verify that the newly loaded function works.

In [ ]:
fadd1 = tvm.runtime.load_module(os.path.join(tmp_path, "myadd.so"))
if tgt.kind.name == "cuda":
    fadd1_dev = tvm.runtime.load_module(os.path.join(tmp_path, "myadd.ptx"))
    fadd1.import_module(fadd1_dev)

if tgt.kind.name == "rocm":
    fadd1_dev = tvm.runtime.load_module(os.path.join(tmp_path, "myadd.hsaco"))
    fadd1.import_module(fadd1_dev)

if tgt.kind.name.startswith("opencl"):
    fadd1_dev = tvm.runtime.load_module(os.path.join(tmp_path, "myadd.cl"))
    fadd1.import_module(fadd1_dev)

fadd1(a, b, c)
tvm.testing.assert_allclose(c.numpy(), a.numpy() + b.numpy())